# Napari-serverkit

`napari-serverkit` is the companion Napari plugin to the [Imaging Server Kit]() project, which aims to [...]

Implementing algorithm servers using this library can greatly enhance your productivity as a developer...
Ideal for quick development of interactive UI.
No need to create a separate, fully dedicated plugin for most applications.
It enhances your experience of creating plugins, without needing to touch `PyQt`.
Compatible with server/client; can be a way of processing images remotely or to overcome installation challenges.
- Provides advanced features including **threading** and **auto_call** on individual parameters.

- The experience of writing algorithm servers is similar but different from `magicgui`, and goes beyond it and beyond Napari alone.

## This tutorial

This tutorial will introduce you to the main functionailties of the plugin. You will learn how to:

- Use `AlgorithmWidget` to 
- 

## A simple example

First, let's consider the simple application of a Gaussian filter to a grayscale image. We'll use the filter implemeted in Scikit-image, although that's not very important. The main parameter of the Gaussian filter is the value of `sigma`, the scale of the Gaussian kernel.

In [ ]:
import skimage.data
from skimage.filters import gaussian

image = skimage.data.coins()
print(f"Input image: Shape={image.shape}, Max={image.max():.02f}, Min={image.min():.02f}")

filtered_image = gaussian(image, sigma=2.0, preserve_range=True)
print(f"Filtered image: Shape={filtered_image.shape}, Max={filtered_image.max():.02f}, Min={filtered_image.min():.02f}")

Now, let's create an algorithm server for our image processing function. You can learn more about how to do this in the [serverkit docs]().

Start by importing the `imaging-server-kit` library, which is a dependency of `napari-serverkit`.

In [ ]:
import imaging_server_kit as sk

print(f"Imaging Server Kit Version: {sk.__version__}")

If you have the library installed, you can create an algorithm server for the gaussian kernel:

In [ ]:
@sk.algorithm_server(
    algorithm_name="gaussian-filter",
    parameters={
        "image": sk.ImageUI(), 
        "sigma": sk.FloatUI("Sigma", default=1.0)
    },
    sample_images=[skimage.data.coins()],
)
def sk_gaussian_filter(image, sigma):
    return [(gaussian(image, sigma=sigma), {"name": "Filtered"}, "image")]

The algorithm server definition lets you interact with the algorithm in interesting ways.

For example, you can access the sample image with:

In [ ]:
sample_image = sk_gaussian_filter.load_sample_images(first_only=True)

sample_image.shape

You could run the Gaussian filter with:

In [ ]:
result_tuple = sk_gaussian_filter.run_algorithm(image=sample_image, sigma=1.0)
first_result = result_tuple[0]
filtered_image = first_result[0]

filtered_image.shape

Now, here is the fun part: what if you wanted to use this algorithm in Napari? Let's do it.

First, you can create a Napari viewer the regular way:

In [ ]:
import napari

viewer = napari.Viewer()

Then, you can use `AlgorithmWidget` to create a dock widget for Napari, based on your algorithm server. You can add the dock widget to the viewer.

In [ ]:
from napari_serverkit import AlgorithmWidget

widget = AlgorithmWidget(viewer, sk_gaussian_filter)

viewer.window.add_dock_widget(widget)

That's it! You can use the gaussian filter interactively in this way.

## Going further

But we can do better! Let's slightly change our algorithm server definition and add an `auto_call` flag to the sigma value.

In [ ]:
@sk.algorithm_server(
    algorithm_name="gaussian-filter",
    parameters={
        "image": sk.ImageUI(), 
        "sigma": sk.FloatUI(default=1.0, auto_call=True)  # Add `auto_call=True`
    },
    sample_images=[skimage.data.coins()],
)
def sk_gaussian_autocalled(image, sigma):
    return [(gaussian(image, sigma=sigma), {"name": "Filtered"}, "image")]

# Add the new widget
viewer.window.add_dock_widget(
    AlgorithmWidget(viewer, sk_gaussian_autocalled), name="Auto-called"
)

Pretty cool, eh?

Here's an even cooler feature: what if you wanted to apply the Gaussian filter tile-by-tile, for example on a large image?

You can do this using the streaming capabilities of `imaging-server-kit` and a tile generator utility function.

In [ ]:
@sk.algorithm_server(
    algorithm_name="gaussian-filter",
    parameters={
        "image": sk.ImageUI(), 
        "sigma": sk.FloatUI(default=1.0, auto_call=True),
    },
    sample_images=[skimage.data.coins()],
)
def sk_gaussian_tiled(image, sigma):
    for image_tile, tile_meta in sk.image_tile_generator_2D(
        image,
        tile_size_px=32,  # Tiles of size 32x32 px
        randomize=True,  # Process the tiles in random order
    ):
        yield [  # yield instead of return
            (
                gaussian(image_tile, sigma=sigma),  # Apply the filter to the image tile
                {
                    "name": "Filtered",
                    "contrast_limits": [0, 255],
                }
                | tile_meta,  # ⬅️ Add the tile metadata here
                "image",
            )
        ]

# Add the new widget
viewer.window.add_dock_widget(
    AlgorithmWidget(viewer, sk_gaussian_tiled), name="Tiled"
)

Notice how, with `auto-call=True`...

In [ ]:
@sk.algorithm_server(
    algorithm_name="gaussian-filter",
    parameters={
        "image": sk.ImageUI(), 
        "sigma": sk.FloatUI(default=1.0, auto_call=True),
    },
    sample_images=[skimage.data.brain()],
)
def sk_gaussian_tiled(image, sigma):
    for image_tile, tile_meta in sk.image_tile_generator_3D(
        image,
        tile_size_px=32,
        randomize=True,
        delay_sec=0.1  # Wait for 100 ms between each streamed tile so that the UI has enough time to render them
    ):
        yield [
            (
                gaussian(image_tile, sigma=sigma),
                {
                    "name": "Filtered",
                }
                | tile_meta,  # ⬅️ Add the tile meta
                "image",
            )
        ]

# Add the new widget
viewer.window.add_dock_widget(
    AlgorithmWidget(viewer, sk_gaussian_tiled), name="Tiled 3D"
)